In [3]:
# Installations/imports
try:
    import requests
except ModuleNotFoundError:
    !pip install requests
    import requests

try:
    import pandas as pd
except ModuleNotFoundError:
    !pip install pandas
    import pandas as pd

try:
    from KalshiClientsBaseV2 import ExchangeClient
    from urllib3.exceptions import HTTPError
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure that KalshiClientsBaseV2 is installed correctly.")
    # If it's a package you can install, you might want to try installing it here
    # !pip install kalshi-python  # Uncomment this line if you need to install the package

import time
import json
import uuid
from datetime import datetime, timedelta
from dateutil import parser

# Set up account credentials and initialize the ExchangeClient
demo_email = "sungjooyoon@college.harvard.edu" # change these to be your personal credentials
demo_password = "****************" # (for extra security, we recommend using a config file)
demo_api_base = "https://demo-api.kalshi.co/trade-api/v2"

exchange_client = ExchangeClient(exchange_api_base=demo_api_base, email=demo_email, password=demo_password)
print(exchange_client.get_exchange_status())

/Users/sungjooyoon/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'exchange_active': True, 'trading_active': True}


In [4]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

NameError: name 'markets_response' is not defined

In [5]:
def get_all_markets(exchange_client):
    try:
        markets_response = exchange_client.get_markets()
        return markets_response.get('markets', [])
    except Exception as e:
        print(f"Error fetching markets: {e}")
        return []

def get_event_data(event_ticker, exchange_client):
    try:
        event_params = {'event_ticker': event_ticker}
        event_response = exchange_client.get_event(**event_params)
        return event_response
    except Exception as e:
        print(f"Error fetching event data for ticker {event_ticker}: {e}")
        return None

def filter_markets(markets):
    filtered_markets = []
    start_date = datetime(2023, 6, 1)
    end_date = datetime(2024, 6, 1)
    
    for market in markets:
        close_time = parser.parse(market['close_time'])
        if market['status'] == 'finalized' and start_date <= close_time <= end_date:
            filtered_markets.append(market)
    
    return filtered_markets

def process_event_data(event_response):
    event_data = []
    event = event_response.get('event', {})
    markets = event_response.get('markets', [])

    for market in markets:
        market_id = market['ticker']
        resolve_date = parser.parse(market['close_time'])
        first_date = resolve_date - timedelta(days=30)
        comp_percentage = [round((43200 - i * 30) / 43200, 5) for i in range(1440)]  # every 30 minutes for 30 days
        comp_category = ['zero_to_ten_percent', 'ten_to_twenty_percent', 'twenty_to_thirty_percent',
                         'thirty_to_forty_percent', 'forty_to_fifty_percent', 'fifty_to_sixty_percent',
                         'sixty_to_seventy_percent', 'seventy_to_eighty_percent', 'eighty_to_ninety_percent',
                         'ninety_to_hundred_percent']

        for comp in range(1440):
            comp_time = first_date + timedelta(minutes=comp * 30)
            comp_index = int(comp_percentage[comp] * 10)
            if comp_index >= len(comp_category):
                comp_index = len(comp_category) - 1

            for option in ["yes", "no"]:
                price = market['yes_ask'] if option == "yes" else market['no_ask']
                outcome = 1 if market['result'] == option else 0
                yes_or_no = 1 if option == "yes" else 0

                event_data.append({
                    'name': event.get('title', ''),
                    'market_id': market_id,
                    'option_id': str(uuid.uuid4()),
                    'category': event.get('category', ''),
                    'choice': market.get('subtitle', ''),
                    'yes_or_no': yes_or_no,
                    'outcome': outcome,
                    'placed_date': comp_time.strftime('%Y-%m-%d %H:%M:%S'),
                    'resolve_date': resolve_date.strftime('%Y-%m-%d %H:%M:%S'),
                    'first_date': first_date.strftime('%Y-%m-%d %H:%M:%S'),
                    'price': price,
                    'comp_percentage': comp_percentage[comp],
                    'comp_category': comp_category[comp_index],
                    'minutes_before': 43200 - comp * 30
                })
    return event_data

def main():
    markets = get_all_markets(exchange_client)
    filtered_markets = filter_markets(markets)
    all_event_data = []

    for market in filtered_markets:
        event_ticker = market.get('event_ticker', '')
        if event_ticker:
            event_response = get_event_data(event_ticker, exchange_client)
            if event_response:
                event_data = process_event_data(event_response)
                all_event_data.extend(event_data)

    if all_event_data:
        df = pd.DataFrame(all_event_data)
        df.to_csv('scraped_data.csv', index=False)
        print("Data scraping complete. The data has been saved to 'scraped_data.csv'.")
    else:
        print("No data to save.")

if __name__ == "__main__":
    main()

No data to save.
